In [2]:
from databases import BDS,Crm,BDSMem,PosicaoDm1, Boletador
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from funcoes_datas import FuncoesDatas
import pandas as pd
from emailer import Email, EmailLer
from pretty_html_table import build_table
from objetos import Fundo

[base_dados.py directory]: C:\Temp\Foundation
[databases.py directory]: C:\Temp\Foundation


In [3]:
##Puxa as classes do databases (consultas nas bases sql da empresa)
crm = Crm()
crm_fundos = Crm(load_cadfundos=True)
bds = BDS()
fdt = FuncoesDatas()
bol = Boletador()
posicao = PosicaoDm1

#retorna posição de todas as carteiras + fundos
posicao = PosicaoDm1()

#Retorna a data de hoje
hoje = bds.banco.hoje()

#Retorna as informações dos fundos
fundos = crm_fundos.fundo_cadastro()

#Retorna todos os produtos da casa
produtos= crm.product()

#Retorna a data 1 do mes atual
hoje = bds.banco.hoje()
dia = hoje + relativedelta(months=-1)
mes,ano = hoje.month,hoje.year

#Traz o primeiro dia util do mes anterior
data_inicial_bol = datetime(ano,mes,1)

#Trazendo o cadastro
cadastro= posicao.po_cadastro_all()[['GuidContaCRM','NomeContaCRM','Tipo','CodigoProduto','Administrador','Custodiante','IdBDS','CNPJ','Officer','Controller','Segmento','TipoCliente']]

In [4]:
cadastro

,GuidContaCRM,NomeContaCRM,Tipo,CodigoProduto,Administrador,Custodiante,IdBDS,CNPJ,Officer,Controller,Segmento,TipoCliente
0,3687feec-e858-e211-a10b-000c29c91374,_(INATIVO) Captalys Mirante FIM - Crédito Privado,Fundo,236,ND,ND,59922557,17012208000123,Paulo Miguel,Luiz Amaral,Fundos,Produto
1,f7a5b490-55b6-ea11-b3a7-005056912b96,_1590 Stork FIM CrPr IE,Fundo,373481,BEM DTVM Ltda,Banco Bradesco SA,59935946,29734102000112,Eduardo Tabone,Diana Silveira,Fundos,Produto
2,6d6d137d-2c06-eb11-b838-005056912b96,_1644 FIM CrPr IE,Fundo,373482,BEM DTVM Ltda,Banco Bradesco SA,59936449,30507573000177,Simone Scivoletto,Daniel Stockler,Fundos,Produto
3,26489ad7-3174-ee11-8108-00505691fa1b,_2094 FIC FIM CrPr IE,Fundo,375822,Santander Caceis Brasil DTVM SA,Santander Caceis Brasil DTVM SA,21426427,29236614000159,Renato Sapuppo,Ivan Preterote de Cunto,Fundos,Produto
4,6f3b5173-bf78-ee11-8108-00505691fa1b,_2549 FIC FIM CrPr IE,Fundo,375833,Santander Caceis Brasil DTVM SA,Santander Caceis Brasil DTVM SA,21438243,38443937000104,Renato Sapuppo,Ivan Preterote de Cunto,Fundos,Produto
...,...,...,...,...,...,...,...,...,...,...,...,...
823,01c3ea23-ed45-ee11-8107-00505691fa1b,_Ype FIM CrPr IE,Fundo,375619,BEM DTVM Ltda,Banco Bradesco SA,21465047,51886989000113,Renato Sapuppo,Ivan Preterote de Cunto,Fundos,Produto
824,42144c9f-61b5-e511-9f22-005056912b96,_Zack FIM CP IE,Fundo,364870,Santander Caceis Brasil DTVM SA,Santander Caceis Brasil DTVM SA,59925826,18599020000196,Bruna Casanova,Bruna Almeida,Fundos,Produto
825,9c445cdf-e3a0-df11-bcd6-0003ffe3d283,_ZAG FIM CP IE,Fundo,250,ND,ND,59911214,09543263000111,Paulo Miguel,Luiz Amaral,Fundos,Produto
826,523f96f8-5a02-eb11-b838-005056912b96,_Zeta FIM CrPr (Monitorado),Fundo,368918,ND,ND,59934449,28454176000132,Paulo Miguel,Luiz Amaral,Fundos,Produto


In [153]:
#retorna apenas o que temos como emissor light

posicao_hoje = posicao.posicao_total_depara(hoje)
posicao_hoje['NomeEmissor'] = posicao_hoje['NomeEmissor'].astype('str').str.lower()
light = posicao_hoje[posicao_hoje['NomeEmissor'].str.contains('light sa')][['NomeContaCRM','NomeProduto','FinanceiroFinal','QuantidadeFinal','QtdeBloq','NomeContaMovimento']]

#Retorna os dados cadastrais
light = pd.merge(left=light,right=cadastro, on='NomeContaCRM',how='left')

#retorna os cotistas que possuem fundo exclusivo

cotistas = posicao_hoje[(posicao_hoje['Classe']=='F Exclusivo')|(posicao_hoje['Classe']=='F Exclusivo Ações')]
cotistas = cotistas[['NomeContaCRM','IdBDS','FinanceiroFinal']].drop_duplicates().rename(columns={'FinanceiroFinal':'Financeiro_Cotista','NomeContaCRM':'Cotista'})

#rertorna os cotistas dos fundos

light = pd.merge(left=light,right=cotistas, on='IdBDS',how='left').drop(columns=['CodigoProduto','IdBDS'])
pls = posicao.fundos_pl_estimado()[['NomeContaCRM','PL']]
light = pd.merge(left=light,right=pls, on='NomeContaCRM',how='left')
light ['Financeiro_Cotista'].loc[light ['Cotista'].isnull()] = light['FinanceiroFinal']
light ['PL'].loc[light ['Cotista'].isnull()] = light['FinanceiroFinal']
light['% do Cotista'] = (light['Financeiro_Cotista']/light['PL'])*100

In [154]:
#Retorna Estrutura ideal de contas

contas = crm.contas_movimento_all()[['accountid','Banco','new_agencia','new_numeroconta','new_digitoconta','ControleJBBName','NomeContaMovimento']]
contas = contas[(contas['NomeContaMovimento'].str.contains('AGORA AÇÕES'))|(contas['Banco'].str.contains('Banco BTG Pactual SA'))].rename(columns={'accountid':'GuidContaCRM'})
contas['Possui Estrutura de Contas'] = 'Sim'
contas = contas[['GuidContaCRM','Possui Estrutura de Contas']]

In [155]:
light = pd.merge(left=light,right=contas, on='GuidContaCRM',how='left')

In [156]:
light.to_excel('plano_light.xlsx')